### Creating a Model

In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")


### Creating an Angent

In [9]:
from langchain.agents import create_agent
from langchain.messages import HumanMessage

system_prompt = """

You are a science fiction writer, create a space capital city at the users request.

User: What is the capital of mars?
Scifi Writer: Marsialis

User: What is the capital of Venus?
Scifi Writer: Venusovia

"""
agent = create_agent(
  model=model,
  system_prompt=system_prompt
)

question = HumanMessage(content="What's the capital of the moon?")

response = agent.invoke(
  {"messages":[question]}
)
print(response['messages'][1].content)


Scifi Writer: Lunaris


### Streaming output

In [12]:
for token, metadata in agent.stream(
    {"messages": [HumanMessage(content="Tell me all about Luna City, the capital of the Moon")]},
    stream_mode="messages"
):

    # token is a message chunk with token content
    # metadata contains which node produced the token
    
    if token.content:  # Check if there's actual content
        print(token.content, end="", flush=True)  # Print token

Ah, Luna City! A jewel nestled in the Sea of Tranquility, or perhaps clinging to the rim of Tycho Crater – the exact location is a matter of much historical debate, and depends on which faction you ask, of course. But its essence, its very being, is undeniable.

Luna City isn't just a capital; it's a testament to humanity's stubborn refusal to stay tethered to a single cradle. It's the beating heart of Lunar governance, the nexus of commerce, and the beating pulse of a civilization forged in vacuum and starlight.

**The Architecture of Ambition:**

Imagine this: not soaring skyscrapers that pierce a sky, but vast, interconnected **domes and subterranean complexes**. The surface domes, shimmering with integrated solar arrays that catch every precious ray of sunlight, are the public face of Luna City. They house grand plazas, botanical gardens that glow with bio-luminescent flora, and observation decks offering jaw-dropping vistas of Earthrise.

But the real lifeblood of the city flows b

### Structured Output Using Ollama locally with llama3.2

In [10]:
from pydantic import BaseModel
from langchain_ollama import ChatOllama

class CapitalInfo(BaseModel):
    name: str
    location: str
    vibe: str
    economy: str

llm = ChatOllama(model="llama3.2")  # local + free
structured = llm.with_structured_output(CapitalInfo)

result: CapitalInfo = structured.invoke("Create a sci-fi capital city for the Moon.")
print(result)
print(result.name)

name='Lunaria' location='Moon Surface, near Mare Serenitatis' vibe='Cyberpunk futurism meets lunar elegance' economy='Hydrogen and Helium-3 mining and tourism-based'
Lunaria


### Creating a Custom Tools using Python Function + Decorator

In [26]:
from langchain.tools import tool

@tool
def square(x: float) -> float:
    """Calculate the square of a number"""
    return x ** 2


@tool("square_root",description="Calculate the square root of a number")
def tool1(x:float) -> float:
  return x**0.5

print(tool1.invoke({"x": 467}))
print(square_root.invoke({"x":25}))

21.61018278497431
5.0


### Adding Tools to Agents 

In [27]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")

from langchain.agents import create_agent

system_prompt = "You are an arithmetic wizard, use your tools to calculate square root and square of any number"

agent = create_agent(
  model = model,
  tools = [tool1,square],
  system_prompt=system_prompt
)

In [28]:
#Calling agent

from langchain.messages import HumanMessage

question = HumanMessage(content="What is the square of 25 ?")

response = agent.invoke(
  {"messages":[question]}
)

print(response['messages'][-1].content)

The square of 25 is 625.
